In [1]:
!pip install keras
!pip install tensorflow

    100% |████████████████████████████████| 307kB 844kB/s ta 0:00:01    85% |███████████████████████████▎    | 256kB 3.8MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 2.6MB/s ta 0:00:011
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 49.1MB 10kB/s eta 0:00:0110% |                                | 81kB 1.2MB/s eta 0:00:41    3% |█▏                              | 1.7MB 1.3MB/s eta 0:00:36    13% |████▍                           | 6.8MB 2.9MB/s eta 0:00:15    16% |█████▎                          | 8.0MB 1.1MB/s eta 0:00:38    18% |██████                          | 9.3MB 3.4MB/s eta 0:00:12    24% |████████                        | 12.2MB 2.6MB/s eta 0:00:15    28% |█████████                       | 13.9MB 1.8MB/s eta 0:00:20    28% |█████████▏                      | 14.1MB 2.6MB/s eta 0:00:14    29% |█████████▌                    

    100% |████████████████████████████████| 890kB 791kB/s eta 0:00:01
    100% |████████████████████████████████| 327kB 1.5MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 2.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for gast ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/9a/1f/0e/3cde98113222b853e98fc0a8e9924480a3e25f1b4008cedb4f
  Running setup.py bdist_wheel for absl-py ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/a0/f8/e9/1933dbb3447ea6ef57062fd5461cb118deb8c2ed074e8344bf
  Running setup.py bdist_wheel for termcolor ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Running setup.py bdist_wheel for html5lib ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built gast absl-py termcolor html5lib
  Found existing installation: numpy 1.12.1
    Uninstalling numpy-1.12.1:
   

In [2]:
import numpy as np
np.random.seed(13)

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

import gensim

Using TensorFlow backend.


In [3]:
path=get_file('alice.txt', origin='http://www.gutenberg.org/files/11/11-0.txt')
corpus = open(path).readlines()[:300]
corpus = [sentence for sentence in corpus if sentence.count(' ') >= 2]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1
dim = 100
window_size = 2


180224/173595 [===============================] - 1s 4us/step


In [4]:
def generate_data(corpus, window_size, V):
    maxlen = window_size*2
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            contexts = []
            labels   = []            
            s = index - window_size
            e = index + window_size + 1
            
            contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
            labels.append(word)

            x = sequence.pad_sequences(contexts, maxlen=maxlen)
            y = np_utils.to_categorical(labels, V)
            yield (x, y)

In [5]:
cbow = Sequential()
cbow.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow.add(Dense(V, activation='softmax'))

In [6]:
cbow.layers

In [7]:
cbow.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [8]:
corpus[:5]

[[348, 349, 65, 115, 10, 116, 57, 196, 197],
 [22, 117, 66, 17, 1, 67, 8, 350, 351, 29, 38, 352, 4, 27],
 [146, 38, 353, 354, 16, 355, 356, 5, 198, 5, 147, 32],
 [357, 67, 5, 118, 1, 358, 8, 1, 199, 148, 359, 360],
 [27, 22, 117, 32, 361, 29, 362, 148, 363]]

In [9]:
for ite in range(10):
    loss = 0.
    for i, (x, y) in enumerate(generate_data(corpus, window_size, V)):
        if i<4:
            print(' ', i, x, y.shape)
        loss += cbow.train_on_batch(x, y)
    print(i)

    print(ite, loss)

  0 [[  0   0 349  65]] (1, 765)
  1 [[  0 348  65 115]] (1, 765)
  2 [[348 349 115  10]] (1, 765)
  3 [[349  65  10 116]] (1, 765)
2786
0 17412.564601898193
  0 [[  0   0 349  65]] (1, 765)
  1 [[  0 348  65 115]] (1, 765)
  2 [[348 349 115  10]] (1, 765)
  3 [[349  65  10 116]] (1, 765)
2786
1 16152.262689352036
  0 [[  0   0 349  65]] (1, 765)
  1 [[  0 348  65 115]] (1, 765)
  2 [[348 349 115  10]] (1, 765)
  3 [[349  65  10 116]] (1, 765)
2786
2 16024.16857266426
  0 [[  0   0 349  65]] (1, 765)
  1 [[  0 348  65 115]] (1, 765)
  2 [[348 349 115  10]] (1, 765)
  3 [[349  65  10 116]] (1, 765)
2786
3 15933.23012804985
  0 [[  0   0 349  65]] (1, 765)
  1 [[  0 348  65 115]] (1, 765)
  2 [[348 349 115  10]] (1, 765)
  3 [[349  65  10 116]] (1, 765)
2786
4 15825.284683167934
  0 [[  0   0 349  65]] (1, 765)
  1 [[  0 348  65 115]] (1, 765)
  2 [[348 349 115  10]] (1, 765)
  3 [[349  65  10 116]] (1, 765)
2786
5 15722.586059451103
  0 [[  0   0 349  65]] (1, 765)
  1 [[  0 348  65 115

In [10]:
f = open('vectors-cbow.txt' ,'w')
f.write('{} {}\n'.format(V-1, dim))

8

In [11]:
vectors = cbow.get_weights()[0]
for word, i in tokenizer.word_index.items():
    str_vec = ' '.join(map(str, list(vectors[i, :])))
    f.write('{} {}\n'.format(word, str_vec))
f.close()

In [12]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors-cbow.txt', binary=False)

In [13]:
w2v.most_similar(positive=['the'])

[('a', 0.6486507654190063),
 ('one', 0.6460806131362915),
 ('this', 0.597328782081604),
 ('adventures', 0.559715211391449),
 ('alice’s', 0.5597040057182312),
 ('any', 0.5511570572853088),
 ('those', 0.5313930511474609),
 ('miles', 0.5266964435577393),
 ('help', 0.524553120136261),
 ('no', 0.5238304138183594)]

In [14]:
w2v.most_similar(positive=['alice'])

[('she', 0.6103900074958801),
 ('you', 0.6003938317298889),
 ('poor', 0.5684895515441895),
 ('them', 0.5613431334495544),
 ('that', 0.5531876087188721),
 ('eat', 0.5528088212013245),
 ('now', 0.5517069697380066),
 ('dark', 0.548737108707428),
 ('marked', 0.5480719804763794),
 ('thought', 0.5456030368804932)]